In [43]:
import pandas as pd
from bs4 import BeautifulSoup, SoupStrainer
import requests
import time
import re

In [71]:
pd.options.display.max_colwidth=200

In [81]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) \
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}

start_time = time.time()

url = 'https://www.toddland.com/collections/american-dad'

# Make an HTTP GET request to the URL
response = requests.get(url)

# for url in results_pages:
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

if response.status_code != 200:
    raise Exception(f'The status code is not 200! It is {response.status_code}.')

print(f'Total time elapsed: {(time.time() - start_time):.2f} seconds')

Total time elapsed: 0.76 seconds


In [95]:
def remove_keywords(strings, keywords):
    # Iterate over each string in the list
    for i, s in enumerate(strings):
        # Check each keyword
        for keyword in keywords:
            # Find the index of the keyword in the string
            index = s.lower().find(keyword.lower())
            # If the keyword is found, slice the string up to the keyword and stop looking for more keywords
            if index != -1:
                strings[i] = s[:index]
                break
    return strings

# Example usage
strings = ["hello world", "example text keyword and more text", "no Keywords here", "another example keyword"]
keywords = ["keyword", "another"]
result = remove_keywords(strings, keywords)
print(result)

['hello world', 'example text ', 'no ', 'another example ']


In [98]:
# Find all product divs within the section
product_divs = soup.select('section.product-grid div[class^="four columns"]')

# Initialize lists to store data
product_names = []
product_types = []
product_prices = []
product_sold_out = []
product_urls = []
product_images = []



# Iterate over each product div
for product in product_divs:
    # Get product name
    product_name = product.find('h3').text.strip()[13:].strip()
    
    
    product_names.append(re.sub(r'^\s*-\s*', '', product_name))
    

    # Get product category
    if re.findall('pin', product_name, re.IGNORECASE):
        product_types.append('Enamel Pin')
    elif re.findall('tee', product_name, re.IGNORECASE):
        product_types.append('T-Shirt')
    elif re.findall('sticker', product_name, re.IGNORECASE):
        product_types.append('Sticker')
    else:
        product_types.append('T-Shirt?')
    
    # Get product price
    product_price = product.find('h4').text.strip()
    product_prices.append(product_price)
    
    # Get product URL
    product_url = product.find('a')['href']
    product_urls.append('https://www.toddland.com' + product_url)
    
    # Get product image URL
    product_image = product.find('img')['src']
    product_images.append('https:' + re.sub('_large','', product_image))

    # Get Sold Out Status
    if product.find('div'):
        product_avail = [status.text.strip() for status in product.find_all('div')]
    else:
        product_avail ='Available'
    product_sold_out.append(product_avail)
    # <div class="sold-out">Sold Out</div>
    
product_names = remove_keywords(product_names, ["hard enamel pin", "enamel pin", "pin le", "Tee", "sticker", "Clear Die Cut"])

# Create DataFrame
df = pd.DataFrame({
    'Name': product_names,
    'Category': product_types,
    'Price': product_prices,
    'SoldOut?': product_sold_out,
    'URL': product_urls,
    'Image': product_images
})

# Display DataFrame
df.sort_values('Name')

,Name,Category,Price,SoldOut?,URL,Image
4,A shot of B12,T-Shirt,$ 28.00,[Sold Out],https://www.toddland.com/collections/american-dad/products/american-dad-a-shot-of-b12-tee-columbia-blue,https://www.toddland.com/cdn/shop/files/B12shirt.jpg?v=1689175266
27,Bazookasharks Clear Die Cut,Sticker,$ 5.00,Available,https://www.toddland.com/collections/american-dad/products/2021-american-dad-bazookasharks-clear-die-cut-sticker-pre-order-ships-8-15-8-30,https://www.toddland.com/cdn/shop/products/sharkssticker.jpg?v=1626818352
21,Chex LeMeneux Clear Die Cut,Sticker,$ 5.00,Available,https://www.toddland.com/collections/american-dad/products/2021-american-dad-chex-lemeneux-clear-die-cut-sticker-pre-order-ships-8-15-8-30,https://www.toddland.com/cdn/shop/products/chexsticker.jpg?v=1626818464
9,Clip Clop,Enamel Pin,$ 12.00,[Sold Out],https://www.toddland.com/collections/american-dad/products/american-dad-clip-clop-enamel-pin-pickup-only,https://www.toddland.com/cdn/shop/products/AD-pin-clip-clop.jpg?v=1561955276
22,Country Roger,Sticker,$ 5.00,Available,https://www.toddland.com/collections/american-dad/products/2020-american-dad-country-roger-sticker,https://www.toddland.com/cdn/shop/products/countryrogersticker.jpg?v=1595375459
3,Family Swap - evergreen green,T-Shirt?,$ 28.00,Available,https://www.toddland.com/collections/american-dad/products/american-dad-family-swap-evergreen-green,https://www.toddland.com/cdn/shop/files/ADfamilyswag.jpg?v=1689175361
26,Golden Turd,Sticker,$ 5.00,Available,https://www.toddland.com/collections/american-dad/products/2020-american-dad-golden-turd-sticker,https://www.toddland.com/cdn/shop/products/goldenturd.jpg?v=1595375233
14,Good for the Ticker,Enamel Pin,$ 18.00,Available,https://www.toddland.com/collections/american-dad/products/american-dad-good-for-the-ticker-enamel-pins-2-pack-pickup-only,https://www.toddland.com/cdn/shop/products/AD-pin-ticker.jpg?v=1561955332
29,Grilled Cheese Brothers,T-Shirt,$ 15.00$ 28.00,"[Sale, Sold Out]",https://www.toddland.com/collections/american-dad/products/american-dad-grilled-cheese-brothers-tee,https://www.toddland.com/cdn/shop/products/grilledcheesebros_e4f9d2b6-e3b0-4d1e-8670-3d156b78aee0.jpg?v=1658193505
20,Grilled Cheese Brothers Clear Die Cut,Sticker,$ 5.00,Available,https://www.toddland.com/collections/american-dad/products/2021-american-dad-grilled-cheese-brothers-clear-die-cut-sticker-pre-order-ships-8-15-8-30,https://www.toddland.com/cdn/shop/products/cheesesticker.jpg?v=1626818291


In [99]:
df.loc[df['Category'] =='Enamel Pin']

,Name,Category,Price,SoldOut?,URL,Image
5,Laura Vanderbooben,Enamel Pin,$ 12.00,Available,https://www.toddland.com/collections/american-dad/products/american-dad-laura-vanderbooben-hard-enamel-pin,https://www.toddland.com/cdn/shop/files/ADvanderbooben.jpg?v=1689175051
6,Tearjerker,Enamel Pin,$ 12.00,Available,https://www.toddland.com/collections/american-dad/products/american-dad-tearjerker-hard-enamel-pin,https://www.toddland.com/cdn/shop/files/ADtearjerker.jpg?v=1689175096
7,Jeannie Gold,Enamel Pin,$ 12.00,Available,https://www.toddland.com/collections/american-dad/products/american-dad-jeannie-gold-hard-enamel-pin,https://www.toddland.com/cdn/shop/files/ADJeannieGold.jpg?v=1689175133
8,Stanny Spanish,Enamel Pin,$ 12.00,Available,https://www.toddland.com/collections/american-dad/products/american-dad-stanny-spanish-hard-enamel-pin,https://www.toddland.com/cdn/shop/files/ADStannySpanish.jpg?v=1689175163
9,Clip Clop,Enamel Pin,$ 12.00,[Sold Out],https://www.toddland.com/collections/american-dad/products/american-dad-clip-clop-enamel-pin-pickup-only,https://www.toddland.com/cdn/shop/products/AD-pin-clip-clop.jpg?v=1561955276
10,West to Mexico Stan,Enamel Pin,$ 12.00,Available,https://www.toddland.com/collections/american-dad/products/2020-american-dad-west-to-mexico-stan-enamel-pin,https://www.toddland.com/cdn/shop/products/westernstan.jpg?v=1595374043
11,Halloween Buckle,Enamel Pin,$ 12.00,Available,https://www.toddland.com/collections/american-dad/products/2020-american-dad-halloween-buckle-enamel-pin,https://www.toddland.com/cdn/shop/products/buckle.jpg?v=1595374315
12,Sexy Cowardly Lion,Enamel Pin,$ 12.00,Available,https://www.toddland.com/collections/american-dad/products/american-dad-sexy-cowardly-lion-enamel-pin-pickup-only,https://www.toddland.com/cdn/shop/products/AD-pin-lion.jpg?v=1561955242
13,Oscar Gold,Enamel Pin,$ 12.00,Available,https://www.toddland.com/collections/american-dad/products/american-dad-oscar-gold-enamel-pin-pickup-only,https://www.toddland.com/cdn/shop/products/AD-pin-oscar-gold.jpg?v=1561955288
14,Good for the Ticker,Enamel Pin,$ 18.00,Available,https://www.toddland.com/collections/american-dad/products/american-dad-good-for-the-ticker-enamel-pins-2-pack-pickup-only,https://www.toddland.com/cdn/shop/products/AD-pin-ticker.jpg?v=1561955332
